In [1]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

In [2]:
last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

Exploration of ETL

In [3]:
# for date, neo_list in neo_data.items():
#     for neo in neo_list:
#         cursor.execute('''
#                     INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
#                     ''', (
#                         neo['id'],
#                         neo['name'],
#                         neo['is_potentially_hazardous_asteroid'],
#                         neo['is_sentry_object'],
#                         neo['close_approach_data'],
#                         neo['estimated_diameter']
#                     ))
# conn.commit()

ELT

In [30]:
def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

             CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                            (datetime.datetime.today(), json.dumps(neo)))
    conn.commit()

load_data()

Exploration of SQL transformation

In [5]:
# df = pd.read_sql_query('''
#                        select json_extract(data, "$.name") as name,
#                        json_extract(data, "$.id") as ID,
#                        json_extract(data, "$.is_potentially_hazardous_asteroid") as potentially_hazardous
#                        from neo;
#                        ''', conn)

# df["potentially_hazardous"] = df["potentially_hazardous"].astype(bool)

Pandas transformation

In [31]:
df = pd.read_sql_query('''SELECT * FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

json_df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2164207,2164207,164207 (2004 GU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2164207,21.19,True,"[{'close_approach_date': '2023-09-29', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2164207?ap...,0.153658,0.343590,153.657929,343.589575,0.095479,0.213497,504.127081,1127.262423
1,2504800,2504800,504800 (2010 CO1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2504800,21.48,True,"[{'close_approach_date': '2023-09-29', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2504800?ap...,0.134448,0.300635,134.448195,300.635304,0.083542,0.186806,441.103017,986.336330
2,3401428,3401428,(2008 CL1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3401428,19.49,False,"[{'close_approach_date': '2023-09-29', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3401428?ap...,0.336167,0.751692,336.166921,751.692087,0.208884,0.467080,1102.909882,2466.181468
3,3748455,3748455,(2016 FA13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3748455,22.50,False,"[{'close_approach_date': '2023-09-29', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3748455?ap...,0.084053,0.187949,84.053340,187.948982,0.052228,0.116786,275.765561,616.630540
4,3773752,3773752,(2017 GK4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3773752,21.79,True,"[{'close_approach_date': '2023-09-29', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3773752?ap...,0.116561,0.260639,116.561459,260.639347,0.072428,0.161954,382.419499,855.115995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,3843536,3843536,(2019 PM2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843536,21.83,False,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3843536?ap...,0.114434,0.255882,114.433973,255.882143,0.071106,0.158998,375.439556,839.508369
185,3989291,3989291,(2020 BD13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3989291,23.30,False,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3989291?ap...,0.058151,0.130029,58.150704,130.028927,0.036133,0.080796,190.783156,426.604105
186,54051300,54051300,(2020 QG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54051300,21.32,True,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54051300?a...,0.144729,0.323623,144.728805,323.623447,0.089930,0.201090,474.832053,1061.756748
187,54115824,54115824,(2021 CN5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54115824,27.80,False,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54115824?a...,0.007321,0.016370,7.320740,16.369672,0.004549,0.010172,24.018176,53.706275


Extracting nested close approach data

In [7]:
json_df['close_approach_data'][0]

[{'close_approach_date': '2023-09-29',
  'close_approach_date_full': '2023-Sep-29 19:07',
  'epoch_date_close_approach': 1696014420000,
  'relative_velocity': {'kilometers_per_second': '7.2527053558',
   'kilometers_per_hour': '26109.7392808454',
   'miles_per_hour': '16223.5766103675'},
  'miss_distance': {'astronomical': '0.2276404891',
   'lunar': '88.5521502599',
   'kilometers': '34054532.295118217',
   'miles': '21160505.1425247146'},
  'orbiting_body': 'Earth'}]

Converting list of dict to dict

In [8]:
for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}

json_df

C:\Users\Jack\AppData\Local\Temp\ipykernel_6880\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2164207,2164207,164207 (2004 GU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2164207,21.190,True,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2164207?ap...,0.153658,0.343590,153.657929,343.589575,0.095479,0.213497,504.127081,1127.262423
1,2504800,2504800,504800 (2010 CO1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2504800,21.480,True,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2504800?ap...,0.134448,0.300635,134.448195,300.635304,0.083542,0.186806,441.103017,986.336330
2,3401428,3401428,(2008 CL1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3401428,19.490,False,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3401428?ap...,0.336167,0.751692,336.166921,751.692087,0.208884,0.467080,1102.909882,2466.181468
3,3748455,3748455,(2016 FA13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3748455,22.500,False,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3748455?ap...,0.084053,0.187949,84.053340,187.948982,0.052228,0.116786,275.765561,616.630540
4,3773752,3773752,(2017 GK4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3773752,21.790,True,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3773752?ap...,0.116561,0.260639,116.561459,260.639347,0.072428,0.161954,382.419499,855.115995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,54205444,54205444,(2021 TD7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54205444,26.660,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54205444?a...,0.012375,0.027672,12.375278,27.671964,0.007690,0.017195,40.601308,90.787285
76,54265773,54265773,(2022 FX1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54265773,25.780,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54265773?a...,0.018559,0.041499,18.559017,41.499224,0.011532,0.025786,60.889166,136.152315
77,54292217,54292217,(2022 OC3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54292217,28.370,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54292217?a...,0.005631,0.012590,5.630604,12.590413,0.003499,0.007823,18.473110,41.307131
78,54312818,54312818,(2022 SL49),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312818,22.870,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54312818?a...,0.070885,0.158504,70.885103,158.503910,0.044046,0.098490,232.562682,520.025967


In [9]:
cad_df = pd.json_normalize(json_df['close_approach_data'])

cad_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,2023-09-29,2023-Sep-29 19:07,1696014420000,Earth,7.2527053558,26109.7392808454,16223.5766103675,0.2276404891,88.5521502599,34054532.295118217,21160505.1425247146
1,2023-09-29,2023-Sep-29 08:36,1695976560000,Earth,24.6069200201,88584.9120722333,55043.219392883,0.4784564686,186.1195662854,71576068.590281882,44475306.6746266916
2,2023-09-29,2023-Sep-29 12:14,1695989640000,Earth,18.3151670092,65934.6012329969,40969.197082803,0.2803678024,109.0630751336,41942426.055620888,26061815.0485407344
3,2023-09-29,2023-Sep-29 09:14,1695978840000,Earth,10.2859294937,37029.3461773945,23008.5956845038,0.2741123129,106.6296897181,41006618.150613523,25480330.9801436974
4,2023-09-29,2023-Sep-29 08:31,1695976260000,Earth,9.1007503926,32762.7014132638,20357.4685531471,0.2973786897,115.6803102933,44487218.562510939,27643075.7882873982
...,...,...,...,...,...,...,...,...,...,...,...
75,2023-10-04,2023-Oct-04 11:57,1696420620000,Earth,14.5500207327,52380.0746375915,32546.9413768968,0.0666464037,25.9254510393,9970160.036680119,6195170.1729348822
76,2023-10-04,2023-Oct-04 08:25,1696407900000,Earth,9.9213800173,35716.9680621602,22193.1349606239,0.0512387614,19.9318781846,7665209.566878218,4762940.3643786884
77,2023-10-04,2023-Oct-04 12:40,1696423200000,Earth,3.1754061142,11431.4620109647,7103.0659367575,0.0960587815,37.3668660035,14370189.107195405,8929221.457710389
78,2023-10-04,2023-Oct-04 04:23,1696393380000,Earth,10.8579552702,39088.6389725962,24288.1601438334,0.3461188578,134.6402356842,51778643.893712886,32173757.3985983868


In [10]:
df = pd.concat([json_df, cad_df, df], axis=1)

df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,...,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles,date_added,data
0,2164207,2164207,164207 (2004 GU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2164207,21.190,True,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2164207?ap...,0.153658,...,Earth,7.2527053558,26109.7392808454,16223.5766103675,0.2276404891,88.5521502599,34054532.295118217,21160505.1425247146,2023-10-03 16:23:59.566727,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2504800,2504800,504800 (2010 CO1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2504800,21.480,True,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2504800?ap...,0.134448,...,Earth,24.6069200201,88584.9120722333,55043.219392883,0.4784564686,186.1195662854,71576068.590281882,44475306.6746266916,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3401428,3401428,(2008 CL1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3401428,19.490,False,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3401428?ap...,0.336167,...,Earth,18.3151670092,65934.6012329969,40969.197082803,0.2803678024,109.0630751336,41942426.055620888,26061815.0485407344,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3748455,3748455,(2016 FA13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3748455,22.500,False,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3748455?ap...,0.084053,...,Earth,10.2859294937,37029.3461773945,23008.5956845038,0.2741123129,106.6296897181,41006618.150613523,25480330.9801436974,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3773752,3773752,(2017 GK4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3773752,21.790,True,"{'close_approach_date': '2023-09-29', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3773752?ap...,0.116561,...,Earth,9.1007503926,32762.7014132638,20357.4685531471,0.2973786897,115.6803102933,44487218.562510939,27643075.7882873982,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,54205444,54205444,(2021 TD7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54205444,26.660,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54205444?a...,0.012375,...,Earth,14.5500207327,52380.0746375915,32546.9413768968,0.0666464037,25.9254510393,9970160.036680119,6195170.1729348822,2023-10-04 16:27:23.528688,{'links': {'self': 'http://api.nasa.gov/neo/re...
76,54265773,54265773,(2022 FX1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54265773,25.780,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54265773?a...,0.018559,...,Earth,9.9213800173,35716.9680621602,22193.1349606239,0.0512387614,19.9318781846,7665209.566878218,4762940.3643786884,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...
77,54292217,54292217,(2022 OC3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54292217,28.370,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54292217?a...,0.005631,...,Earth,3.1754061142,11431.4620109647,7103.0659367575,0.0960587815,37.3668660035,14370189.107195405,8929221.457710389,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...
78,54312818,54312818,(2022 SL49),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312818,22.870,False,"{'close_approach_date': '2023-10-04', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54312818?a...,0.070885,...,Ear

Cleaning up df

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 30 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    80 non-null     object 
 1   neo_reference_id                                      80 non-null     object 
 2   name                                                  80 non-null     object 
 3   nasa_jpl_url                                          80 non-null     object 
 4   absolute_magnitude_h                                  80 non-null     float64
 5   is_potentially_hazardous_asteroid                     80 non-null     bool   
 6   close_approach_data                                   80 non-null     object 
 7   is_sentry_object                                      80 non-null     bool   
 8   links.self                                            80 non-n

In [12]:
df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
        'estimated_diameter.kilometers.estimated_diameter_min',
        'estimated_diameter.kilometers.estimated_diameter_max',
        'estimated_diameter.miles.estimated_diameter_min',
        'estimated_diameter.miles.estimated_diameter_max',
        'estimated_diameter.feet.estimated_diameter_min',
        'estimated_diameter.feet.estimated_diameter_max',
        'close_approach_date', 'epoch_date_close_approach',
        'relative_velocity.miles_per_hour',
        'miss_distance.miles'], axis=1
        )

df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,date_added,data
0,2164207,164207 (2004 GU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2164207,21.190,True,False,153.657929,343.589575,2023-Sep-29 19:07,Earth,7.2527053558,26109.7392808454,0.2276404891,88.5521502599,34054532.295118217,2023-10-03 16:23:59.566727,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2504800,504800 (2010 CO1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2504800,21.480,True,False,134.448195,300.635304,2023-Sep-29 08:36,Earth,24.6069200201,88584.9120722333,0.4784564686,186.1195662854,71576068.590281882,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3401428,(2008 CL1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3401428,19.490,False,False,336.166921,751.692087,2023-Sep-29 12:14,Earth,18.3151670092,65934.6012329969,0.2803678024,109.0630751336,41942426.055620888,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3748455,(2016 FA13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3748455,22.500,False,False,84.053340,187.948982,2023-Sep-29 09:14,Earth,10.2859294937,37029.3461773945,0.2741123129,106.6296897181,41006618.150613523,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3773752,(2017 GK4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3773752,21.790,True,False,116.561459,260.639347,2023-Sep-29 08:31,Earth,9.1007503926,32762.7014132638,0.2973786897,115.6803102933,44487218.562510939,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,54205444,(2021 TD7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54205444,26.660,False,False,12.375278,27.671964,2023-Oct-04 11:57,Earth,14.5500207327,52380.0746375915,0.0666464037,25.9254510393,9970160.036680119,2023-10-04 16:27:23.528688,{'links': {'self': 'http://api.nasa.gov/neo/re...
76,54265773,(2022 FX1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54265773,25.780,False,False,18.559017,41.499224,2023-Oct-04 08:25,Earth,9.9213800173,35716.9680621602,0.0512387614,19.9318781846,7665209.566878218,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...
77,54292217,(2022 OC3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54292217,28.370,False,False,5.630604,12.590413,2023-Oct-04 12:40,Earth,3.1754061142,11431.4620109647,0.0960587815,37.3668660035,14370189.107195405,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...
78,54312818,(2022 SL49),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312818,22.870,False,False,70.885103,158.503910,2023-Oct-04 04:23,Earth,10.8579552702,39088.6389725962,0.3461188578,134.6402356842,51778643.893712886,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [13]:
df = df.rename(columns={
    'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
    'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
    'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
    'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
    'miss_distance.astronomical': 'miss_distance_astronomical',
    'miss_distance.lunar': 'miss_distance_lunar',
    'miss_distance.kilometers': 'miss_distance_kilometers',
    })
df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
0,2164207,164207 (2004 GU9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2164207,21.190,True,False,153.657929,343.589575,2023-Sep-29 19:07,Earth,7.2527053558,26109.7392808454,0.2276404891,88.5521502599,34054532.295118217,2023-10-03 16:23:59.566727,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,2504800,504800 (2010 CO1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2504800,21.480,True,False,134.448195,300.635304,2023-Sep-29 08:36,Earth,24.6069200201,88584.9120722333,0.4784564686,186.1195662854,71576068.590281882,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3401428,(2008 CL1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3401428,19.490,False,False,336.166921,751.692087,2023-Sep-29 12:14,Earth,18.3151670092,65934.6012329969,0.2803678024,109.0630751336,41942426.055620888,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3748455,(2016 FA13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3748455,22.500,False,False,84.053340,187.948982,2023-Sep-29 09:14,Earth,10.2859294937,37029.3461773945,0.2741123129,106.6296897181,41006618.150613523,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3773752,(2017 GK4),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3773752,21.790,True,False,116.561459,260.639347,2023-Sep-29 08:31,Earth,9.1007503926,32762.7014132638,0.2973786897,115.6803102933,44487218.562510939,2023-10-03 16:23:59.567725,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,54205444,(2021 TD7),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54205444,26.660,False,False,12.375278,27.671964,2023-Oct-04 11:57,Earth,14.5500207327,52380.0746375915,0.0666464037,25.9254510393,9970160.036680119,2023-10-04 16:27:23.528688,{'links': {'self': 'http://api.nasa.gov/neo/re...
76,54265773,(2022 FX1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54265773,25.780,False,False,18.559017,41.499224,2023-Oct-04 08:25,Earth,9.9213800173,35716.9680621602,0.0512387614,19.9318781846,7665209.566878218,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...
77,54292217,(2022 OC3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54292217,28.370,False,False,5.630604,12.590413,2023-Oct-04 12:40,Earth,3.1754061142,11431.4620109647,0.0960587815,37.3668660035,14370189.107195405,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...
78,54312818,(2022 SL49),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54312818,22.870,False,False,70.885103,158.503910,2023-Oct-04 04:23,Earth,10.8579552702,39088.6389725962,0.3461188578,134.6402356842,51778643.893712886,2023-10-04 16:27:23.529686,{'links': {'self': 'http://api.nasa.gov/neo/re...


Adding date awareness to allow for updating data without duplicates and loading previous data

In [14]:
datetime.date.today() - datetime.timedelta(days=365)

datetime.date(2022, 10, 4)

In [32]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = datetime.datetime.strptime(f.read(), '%Y-%m-%d').date()
except FileNotFoundError:
        last_execution_date = (datetime.date.today() - datetime.timedelta(days=365))

last_execution_date

datetime.date(2023, 10, 4)

In [16]:
delta = (datetime.date.today() - last_execution_date).days
delta

0

In [17]:
# while delta % 7 != 0:
#     variable = (datetime.date.today() - datetime.timedelta(delta%7))
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - delta % 7
#     print(link)
# while delta // 7 != 0:
#     variable = variable - datetime.timedelta(days=7)
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - 7
#     print(link)

In [18]:
delta = 8

for x in range (0, delta, 7):
    print(x)

0
7


In [19]:
delta = 14
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta == 7:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)
    else:
        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    return links

get_links() 
        

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-27&end_date=2023-10-03&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-20&end_date=2023-09-26&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [20]:
delta = 7
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    if delta % 7 == 0:
        # end_date = datetime.date.today()
        # start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        # end_date = end_date.strftime('%Y-%m-%d')
        # link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        # links.append(link)

        for x in range(0, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)  
        
    return links

get_links()

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-26&end_date=2023-10-03&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [21]:
delta = 15

for x in range (0, delta, 7):
    print(x)

def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (datetime.date.today() - datetime.timedelta(days=delta%7-1)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)


    for x in range(delta%7, delta, 7):
        end_date = (datetime.date.today() - datetime.timedelta(days=x)).strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=x+6)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    return links

get_links() 

0
7
14


['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-10-04&end_date=2023-10-04&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-27&end_date=2023-10-03&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-20&end_date=2023-09-26&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [39]:
delta = 14
def get_links(delta):
    if delta == 0:
        return []

    links = []

    def build_link(x, y):
        end_date = x.strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=y)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta % 7 != 0:
        build_link(datetime.date.today(), delta%7-1)
    

    for x in range(delta%7, delta, 7):
        build_link((datetime.date.today() - datetime.timedelta(days=x)), x+6)

    return links

get_links(delta) 

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-28&end_date=2023-10-04&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-21&end_date=2023-09-27&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [43]:
links = get_links(delta)

def extract_load(links):
    
    for link in links:
    
        response = requests.get(link)
        data = response.json()
        neo_data = data['near_earth_objects']

        cursor = conn.cursor()

        cursor.executescript('''

            DROP TABLE IF EXISTS neo;                   

            CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

        for date, neo_list in neo_data.items():
            for neo in neo_list:
                cursor.execute('INSERT INTO neo VALUES (?, ?)',
                               (datetime.datetime.today(), json.dumps(neo)))
        conn.commit()

        with open('last_executed.txt', 'w') as f:
            f.write(str(datetime.date.today()))

extract_load(links)

OperationalError: near "CREATE": syntax error

In [24]:
def transform():
    df = pd.read_sql_query('''SELECT data FROM neo''', conn)

    df['data'] = df['data'].apply(json.loads)

    json_df = pd.json_normalize(df['data'])

    for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]
                                             for k, v in element.items()}

    cad_df = pd.json_normalize(json_df['close_approach_data'])

    df = pd.concat([json_df, cad_df, df], axis=1)

    df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
                  'estimated_diameter.kilometers.estimated_diameter_min',
                  'estimated_diameter.kilometers.estimated_diameter_max',
                  'estimated_diameter.miles.estimated_diameter_min',
                  'estimated_diameter.miles.estimated_diameter_max',
                  'estimated_diameter.feet.estimated_diameter_min',
                  'estimated_diameter.feet.estimated_diameter_max',
                  'close_approach_date', 'epoch_date_close_approach',
                  'relative_velocity.miles_per_hour',
                  'miss_distance.miles'], axis=1
                 )

    df = df.rename(columns={
        'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
        'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
        'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
        'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
        'miss_distance.astronomical': 'miss_distance_astronomical',
        'miss_distance.lunar': 'miss_distance_lunar',
        'miss_distance.kilometers': 'miss_distance_kilometers',
        })

    cols = ['relative_velocity_kilometers_per_second',
            'relative_velocity_kilometers_per_hour',
            'miss_distance_astronomical',
            'miss_distance_lunar',
            'miss_distance_kilometers'
            ]

    df[cols] = df[cols].astype('float')
        
    return df

df = transform()

C:\Users\Jack\AppData\Local\Temp\ipykernel_6880\3632851465.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]


In [27]:
cols = ['relative_velocity_kilometers_per_second',
        'relative_velocity_kilometers_per_hour',
        'miss_distance_astronomical',
        'miss_distance_lunar',
        'miss_distance_kilometers'
        ]

df[cols] = df[cols].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 17 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       80 non-null     object        
 1   name                                     80 non-null     object        
 2   nasa_jpl_url                             80 non-null     object        
 3   absolute_magnitude_h                     80 non-null     float64       
 4   is_potentially_hazardous_asteroid        80 non-null     bool          
 5   is_sentry_object                         80 non-null     bool          
 6   estimated_diameter_meters_min            80 non-null     float64       
 7   estimated_diameter_meters_max            80 non-null     float64       
 8   close_approach_date_full                 80 non-null     object        
 9   orbiting_body                            80 n

In [92]:
df_hazard = df[df['is_potentially_hazardous_asteroid']==1]
df_hazard_wow = df_hazard[df_hazard['miss_distance_lunar']<5]
df_hazard_wow

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters_min,estimated_diameter.meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,data
3819,2367789,367789 (2011 AG5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2367789,21.77,True,False,117.639989,263.051013,2023-Feb-03 08:51,Earth,9.918245,35705.682294,0.012145,4.724500,1.816903e+06,{'links': {'self': 'http://api.nasa.gov/neo/re...
4422,3553169,(2010 XC15),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3553169,21.48,True,False,134.448195,300.635304,2022-Dec-27 18:15,Earth,10.099342,36357.631938,0.005163,2.008254,7.723149e+05,{'links': {'self': 'http://api.nasa.gov/neo/re...
4530,54320190,(2022 UD9),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54320190,21.81,True,False,115.492818,258.249791,2022-Dec-22 11:34,Earth,10.269186,36969.068672,0.011889,4.624780,1.778553e+06,{'links': {'self': 'http://api.nasa.gov/neo/re...
5078,3279432,(2005 LW3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3279432,21.67,True,False,123.184191,275.448225,2022-Nov-23 10:06,Earth,13.494442,48579.990412,0.007620,2.964048,1.139885e+06,{'links': {'self': 'http://api.nasa.gov/neo/re...
